<a href="https://colab.research.google.com/github/lucasnserra/Market-Basket-Analysis-in-Python/blob/main/1%C2%B0_Market_Basket_Basic_Association_Rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#Importo Librerias
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import permutations

In [23]:
#Cargo dataset y convierto en lista (busqueda de antecedentes y consecuentes)
transactions = pd.read_csv('https://raw.githubusercontent.com/lucasnserra/Market-Basket-Analysis-in-Python/main/bookstore_transactions.csv')
#transactions = transactions['Transaction'].apply(lambda t: t.split(','))
#transactions = list(transactions)
transactions['Transaction']


0        History|Fiction
1       History|Biogaphy
2         History|Poetry
3       Biogaphy|History
4       Biogaphy|Fiction
              ...       
2717      Poetry|History
2718      Poetry|History
2719      Poetry|History
2720      Poetry|History
2721      Poetry|History
Name: Transaction, Length: 2722, dtype: object

In [24]:
#Utilizo Permutations de Itertools para crear una lista con todos los antecedendes y consecuentes unicos.
#Estas lineas de codigo estan limitadas ya que funciona solamente con 1 antecedente y 1 consecuente. Tambien no limita o elige las mejores permutaciones 
flattened = [i for t in transactions for i in t]
books = list(set(flattened))
rules = list(permutations(books,2))
#Obtengo cantidad de permutaciones distintas   
len(rules)


72

In [25]:
#Aplicacion de reglas  
#-> Metrica (metric) = Medida de rendimiento de una regla
#-> Poda (Pruning) = Utiliza una metrica para descargar reglas

In [26]:
#Aplicacion de Metrica Support 
transactions2  = pd.read_csv('https://raw.githubusercontent.com/lucasnserra/Market-Basket-Analysis-in-Python/main/bookstore_transactions.csv',index_col = False)
transactions1 = transactions2['Transaction'].apply(lambda t: t.split('|'))
transactions1 = list(transactions1)

In [27]:
#importo la libreria necesaria para crear la metrica
from mlxtend.preprocessing import TransactionEncoder
#Creo el encoder y lo entreno
encoder = TransactionEncoder().fit(transactions1)
#Hago el OneHot (Crea una matriz de True o False si aparece el atributo en la fila)
onehot = encoder.transform(transactions1)
#Creo DF 
onehot = pd.DataFrame(onehot,columns=encoder.columns_)
#Obtengo metrica de soporte calculando la media (por cada items)
onehot.mean()
onehot

,Biogaphy,Fiction,History,Poetry
0,False,True,True,False
1,True,False,True,False
2,False,False,True,True
3,True,False,True,False
4,True,True,False,False
...,...,...,...,...
2717,False,False,True,True
2718,False,False,True,True
2719,False,False,True,True
2720,False,False,True,True


In [28]:
#Obtengo metrica de soporte por multiples items
onehot['Poetry+History'] = np.logical_and(onehot['Poetry'],onehot['History'])
onehot.mean() 

Biogaphy          0.458486
Fiction           0.250551
History           0.955915
Poetry            0.335048
Poetry+History    0.303453
dtype: float64

Conclusiones: 
La metrica de Support puede ser sesgada si se trata de manera aislada ya que hay ciertos productos como la leche y el pan que son comunes en las compras y que estos se compren juntos no demuestra un comportamiento por el analisis de X-Sell. Por lo cual se debe sustentar con otras 2 metricas. 
-Metrica de confianza(Confidence Metric).
-Metrica de Elavacion(Lift Metric).


**La métrica de elevación**: se calcula como el soporte de los artículos X e Y dividido por el soporte de X multiplicado por el soporte de Y. El numerador nos da la proporción de transacciones que contienen tanto X como Y. El denominador nos dice cuál sería esa proporción si X e Y fueron asignados aleatoria e independientemente a las transacciones.


In [29]:
# Compute support for Biogaphy and Fiction
supportBF = np.logical_and(onehot['Biogaphy'], onehot['Fiction']).mean()

# Compute support for Biogaphy and History
supportBH = np.logical_and(onehot['Biogaphy'], onehot['History']).mean()

# Compute support for Fiction and Biogaphy
supportFB = np.logical_and(onehot['Fiction'], onehot['Biogaphy']).mean()

# Print support values
print("Biogaphy and Fiction %.2f" % supportBF)
print("Biogaphy and History: %.2f" % supportBH)
print("Fiction and Biogaphy: %.2f" % supportFB)



Biogaphy and Fiction 0.01
Biogaphy and History: 0.43
Fiction and Biogaphy: 0.01


**Metrica de confianza:** se define como el soporte de los artículos X e Y dividido por el soporte del artículo X. La confianza nos dice la probabilidad de que compraremos Y, dado que hemos comprado X.

In [30]:
#Detecte un soporte importante en biografia e historia 
supportBH = np.logical_and(onehot['Biogaphy'], onehot['History']).mean()

# Computo soporte de Biogaphy
supportB = onehot['Biogaphy'].mean()

# Computo soporte de History
supportH = onehot['History'].mean()

# Computo confianza en ambas Reglas
confidenceBH = supportBH / supportB
confidenceHB = supportBH / supportH

# Resultados
print('{0:.2f}, {1:.2f}'.format(supportB, supportH))
print('{0:.2f}, {1:.2f}'.format(confidenceBH, confidenceHB))

0.46, 0.96
0.94, 0.45


In [38]:
#Creo la funcion de Confianza
def Confidence(antecedent, consequent):
  supportAC = np.logical_and(antecedent, consequent).mean()
  # Computo soporte de antecedent
  supportA = antecedent.mean()

# Computo soporte de consequent
  supportC = consequent.mean()

# Computo confianza en ambas Reglas
  confidenceAC = supportAC / supportA
  onfidenceCA = supportAC / supportC
  return print('Confidence: {0:.2f}, {1:.2f}'.format(confidenceBH, confidenceHB))
Confidence(onehot['Biogaphy'],onehot['History'])

Confidence: 0.94, 0.45


Aunque el soporte es idéntico para las dos reglas de asociación, la confianza es mucho mayor para Biogaphy -> history, ya que History tiene un soporte mayor que Biogaphy.

In [32]:
#Creo funcion de conviction
def conviction(antecedent, consequent):
	# Compute support for antecedent AND consequent
	supportAC = np.logical_and(antecedent, consequent).mean()

	# Compute support for antecedent
	supportA = antecedent.mean()

	# Compute support for NOT consequent
	supportnC = 1.0 - consequent.mean()

	# Compute support for antecedent and NOT consequent
	supportAnC = supportA - supportAC

    # Return conviction
	return supportA * supportnC / supportAnC

In [33]:
"""# Compute conviction for twilight -> potter and potter -> twilight
convictionTP = conviction(Biogaphy, potter)
convictionPT = conviction(potter, twilight)

# Compute conviction for twilight -> hunger and hunger -> twilight
convictionTH = conviction(twilight, hunger)
convictionHT = conviction(hunger, twilight)

# Compute conviction for potter -> hunger and hunger -> potter
convictionPH = conviction(potter, hunger)
convictionHP = conviction(hunger,potter)

# Print results
print('Harry Potter -> Twilight: ', convictionHT)
print('Twilight -> Potter: ', convictionTP)"""

"# Compute conviction for twilight -> potter and potter -> twilight\nconvictionTP = conviction(Biogaphy, potter)\nconvictionPT = conviction(potter, twilight)\n\n# Compute conviction for twilight -> hunger and hunger -> twilight\nconvictionTH = conviction(twilight, hunger)\nconvictionHT = conviction(hunger, twilight)\n\n# Compute conviction for potter -> hunger and hunger -> potter\nconvictionPH = conviction(potter, hunger)\nconvictionHP = conviction(hunger,potter)\n\n# Print results\nprint('Harry Potter -> Twilight: ', convictionHT)\nprint('Twilight -> Potter: ', convictionTP)"

In [39]:
#Creo funcion de Lift
def lift(antecedent, consequent):
  supportAC = np.logical_and(antecedent, consequent).mean()
  # Computo soporte de antecedent
  supportA = antecedent.mean()

# Computo soporte de consequent
  supportC = consequent.mean()
# Compute lift
  lift = supportAC / (supportA * supportC)
  return print("Lift: %.2f" % lift)
lift(onehot['Biogaphy'],onehot['History'])

Lift: 0.98


In [34]:
# Define a function to compute Zhang's metric
def zhang(antecedent, consequent):
	# Compute the support of each book
	supportA = antecedent.mean()
	supportC = consequent.mean()

	# Compute the support of both books
	supportAC = np.logical_and(antecedent, consequent).mean()

	# Complete the expressions for the numerator and denominator
	numerator = supportAC - supportA*supportC
	denominator = max(supportAC*(1-supportA), supportA*(supportC-supportAC))

	# Return Zhang's metric
	return numerator / denominator

In [35]:
"""# Define an empty list for Zhang's metric
zhangs_metric = []

# Loop over lists in itemsets
for itemset in itemsets:
    # Extract the antecedent and consequent columns
	antecedent = books[itemset[0]]
	consequent = books[itemset[1]]
    
    # Complete Zhang's metric and append it to the list
	zhangs_metric.append(zhang(antecedent, consequent))
    
# Print results
rules['zhang'] = zhangs_metric
print(rules)"""

"# Define an empty list for Zhang's metric\nzhangs_metric = []\n\n# Loop over lists in itemsets\nfor itemset in itemsets:\n    # Extract the antecedent and consequent columns\n\tantecedent = books[itemset[0]]\n\tconsequent = books[itemset[1]]\n    \n    # Complete Zhang's metric and append it to the list\n\tzhangs_metric.append(zhang(antecedent, consequent))\n    \n# Print results\nrules['zhang'] = zhangs_metric\nprint(rules)"